<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/livia-branch/AmazonDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Selected furniture data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_furniture = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_furniture.show()
print((df_furniture.count(), len(df_furniture.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [4]:
# Selected personal care applianaces
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_personal_care = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_personal_care.show()
print((df_personal_care.count(), len(df_personal_care.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [5]:
# Selected office products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_office = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_office.show()
print((df_office.count(), len(df_office.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [6]:
# Selected apparel products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_apparel = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_apparel.show()
print((df_apparel.count(), len(df_apparel.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [7]:
# Selected home products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_home = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_home.show()
print((df_home.count(), len(df_home.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [8]:
# Merge databases
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

df_merge = unionAll(df_furniture, df_personal_care, df_office, df_apparel, df_home)
df_merge.show()

print((df_merge.count(), len(df_merge.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [12]:
# Only considering verified purchases
df_df_merge = df_merge.filter(df_merge.verified_purchase == "Y")

In [19]:
# Drop the non-beneficial columns
columns_to_drop = ["marketplace","review_id","product_parent","star_rating","helpful_votes","total_votes","vine","review_headline","review_body","verified_purchase"]

df_merge = df_merge.drop(*columns_to_drop)
df_merge.show()

print((df_merge.count(), len(df_merge.columns)))

+-----------+----------+--------------------+----------------+-----------+-----------+
|customer_id|product_id|       product_title|product_category|review_date|review_year|
+-----------+----------+--------------------+----------------+-----------+-----------+
|   24509695|B004HB5E0E|Shoal Creek Compu...|       Furniture| 2015-08-31|       2015|
|   34731776|B0042TNMMS|Dorel Home Produc...|       Furniture| 2015-08-31|       2015|
|    1272331|B0030MPBZ4|Bathroom Vanity T...|       Furniture| 2015-08-31|       2015|
|   45284262|B005G02ESA|Sleep Master Ulti...|       Furniture| 2015-08-31|       2015|
|   30003523|B005JS8AUA|1 1/4" GashGuards...|       Furniture| 2015-08-31|       2015|
|   18311821|B00AVUQQGQ|Serta Bonded Leat...|       Furniture| 2015-08-31|       2015|
|   42943632|B00CFY20GQ|Prepac Shoe Stora...|       Furniture| 2015-08-31|       2015|
|   43157304|B00FKC48QA|HomCom PU Leather...|       Furniture| 2015-08-31|       2015|
|   51918480|B00N9IAL9K|  Folding Step Stoo

In [14]:
# Get datatypes
df_merge.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- review_date: string (nullable = true)



In [15]:
# Convert date from string to datetime
from pyspark.sql.types import *
df_merge = df_merge.withColumn("review_date", df_merge["review_date"].cast(DateType()))

df_merge.printSchema()
df_merge.show()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- review_date: date (nullable = true)

+-----------+----------+--------------------+----------------+-----------+
|customer_id|product_id|       product_title|product_category|review_date|
+-----------+----------+--------------------+----------------+-----------+
|   24509695|B004HB5E0E|Shoal Creek Compu...|       Furniture| 2015-08-31|
|   34731776|B0042TNMMS|Dorel Home Produc...|       Furniture| 2015-08-31|
|    1272331|B0030MPBZ4|Bathroom Vanity T...|       Furniture| 2015-08-31|
|   45284262|B005G02ESA|Sleep Master Ulti...|       Furniture| 2015-08-31|
|   30003523|B005JS8AUA|1 1/4" GashGuards...|       Furniture| 2015-08-31|
|   18311821|B00AVUQQGQ|Serta Bonded Leat...|       Furniture| 2015-08-31|
|   42943632|B00CFY20GQ|Prepac Shoe Stora...|       Furniture| 2015-08-31|
|   43157304|B00FKC48QA|

In [16]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
 
df_merge = df_merge.withColumn("review_year",year(df_merge.review_date))
df_merge.show()

+-----------+----------+--------------------+----------------+-----------+-----------+
|customer_id|product_id|       product_title|product_category|review_date|review_year|
+-----------+----------+--------------------+----------------+-----------+-----------+
|   24509695|B004HB5E0E|Shoal Creek Compu...|       Furniture| 2015-08-31|       2015|
|   34731776|B0042TNMMS|Dorel Home Produc...|       Furniture| 2015-08-31|       2015|
|    1272331|B0030MPBZ4|Bathroom Vanity T...|       Furniture| 2015-08-31|       2015|
|   45284262|B005G02ESA|Sleep Master Ulti...|       Furniture| 2015-08-31|       2015|
|   30003523|B005JS8AUA|1 1/4" GashGuards...|       Furniture| 2015-08-31|       2015|
|   18311821|B00AVUQQGQ|Serta Bonded Leat...|       Furniture| 2015-08-31|       2015|
|   42943632|B00CFY20GQ|Prepac Shoe Stora...|       Furniture| 2015-08-31|       2015|
|   43157304|B00FKC48QA|HomCom PU Leather...|       Furniture| 2015-08-31|       2015|
|   51918480|B00N9IAL9K|  Folding Step Stoo

In [17]:
# Create a table to count data rows per year
row_year_table = df_merge.groupby("review_year").agg({"review_year":"count"}).withColumnRenamed("count(review_year)", "year_count")
row_year_table.sort(row_year_table.review_year.desc()).show(truncate=False)

+-----------+----------+
|review_year|year_count|
+-----------+----------+
|2015       |10692196  |
|2014       |9946608   |
|2013       |5066874   |
|2012       |1889341   |
|2011       |990085    |
|2010       |562017    |
|2009       |358652    |
|2008       |255264    |
|2007       |204335    |
|2006       |78536     |
|2005       |55038     |
|2004       |48632     |
|2003       |44746     |
|2002       |30930     |
|2001       |23299     |
|2000       |15646     |
|1999       |2149      |
|1998       |23        |
|null       |0         |
+-----------+----------+



In [25]:
# Only considering 2015 data for week 1
df_merge = df_merge.filter(df_merge.review_year == "2015")
print((df_merge.count(), len(df_merge.columns)))

(10692196, 6)


In [36]:
# Convert PySpark Dataframe to Pandas DataFrame
df_new = df_merge.toPandas()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34339)
Traceback (most recent call last):
  File "/content/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [ ]:
# Print out the Country value counts
import pandas as pd

product_count = df.product_id.value_counts()
product_count

In [ ]:
# Visualize the value counts
product_count.plot.density()

In [ ]:
# Determine which values to replace
replace_product = list(product_count[product_count < 100].index)

# Replace in DataFrame
for product in replace_product :
    ramen_df.Country = ramen_df.Country.replace(country,"Other")


# Check to make sure binning was successful
ramen_df.Country.value_counts()

In [30]:
product_table = df_merge.groupby("product_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "product_id")
product_table.show()

+----------+----------+
|product_id|product_id|
+----------+----------+
|B00LN17FPI|        95|
|B006H3IAYM|        15|
|B00H4SSFQW|        70|
|B0043JR5NO|       315|
|B0051CMYCO|        85|
|B00GEXJH4W|       784|
|B0103KTOPS|         5|
|B00FRR0BU0|        25|
|B006Z8OID6|         5|
|B00XGUFUGK|        10|
|B005G6Z4OK|        50|
|B00HVW3TXA|       200|
|B0017RAPTQ|       120|
|B001BX5A78|        10|
|B0016L8BD0|        35|
|B00LCIJUQY|         5|
|B004D7X2Y2|        35|
|B007T0LZF8|        35|
|B00BUHWG0O|        30|
|B00570FQRU|        15|
+----------+----------+
only showing top 20 rows



In [26]:
# Create table with customer_IDs and products bought
summary_table = df_merge.groupby("customer_id").pivot("product_id").count()
summary_table.show()

AnalysisException: ignored

In [27]:
# Create a random sample of the database for the algorithym
# Please note that for future iterations we shoudl use stratify samplying


df_sample = df_merge(True, 0.1, 100)
print((df_sample.count(), len(df_sample.columns)))

TypeError: ignored

In [ ]:
# Save to csv file
df_sample.write.csv('df_sample.csv')